In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
import os
import glob

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [28]:
from MyVision.dataset.Dataset import DatasetUtils
from MyVision.engine.Engine import CustomTrainer

In [27]:
class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.lin1 = nn.Linear(in_features=5, out_features=16)
        self.lin2 = nn.Linear(in_features=16, out_features=n_features)
    
    def forward(self, x):
        x = self.lin1(x)
        x = self.lin2(x)
        
        return F.softmax(x)

In [6]:
?DatasetUtils.splitter

Signature:
DatasetUtils.splitter(
    train_df=None,
    image_paths=None,
    targets=None,
    valid_size=0.25,
)
Docstring: <no docstring>
File:      e:\library\myvision\dataset\dataset.py
Type:      function


In [7]:
path = r'E:\Btech project\leaf_disease\feature_files\Balanced_multiclass_features(RGB).csv'

In [8]:
train_df = pd.read_csv(path)

In [9]:
train_df.head()

,Unnamed: 0,contrast,dissimilarity,homogeneity,ASM,energy,label
0,0,0.596856,0.058569,0.993140,0.962980,0.981315,0
1,1,0.249604,0.054572,0.991393,0.971380,0.985586,0
2,2,0.122400,0.030571,0.995105,0.983770,0.991852,0
3,3,0.188586,0.052062,0.990554,0.968699,0.984225,0
4,4,0.225654,0.053860,0.991675,0.970412,0.985095,0


In [10]:
train_df = train_df.iloc[:, 1:]

In [11]:
train_df.head()

,contrast,dissimilarity,homogeneity,ASM,energy,label
0,0.596856,0.058569,0.993140,0.962980,0.981315,0
1,0.249604,0.054572,0.991393,0.971380,0.985586,0
2,0.122400,0.030571,0.995105,0.983770,0.991852,0
3,0.188586,0.052062,0.990554,0.968699,0.984225,0
4,0.225654,0.053860,0.991675,0.970412,0.985095,0


In [12]:
feature_columns = ['contrast','dissimilarity','homogeneity','ASM','energy']

In [13]:
X_train, y_train, X_test, y_test = DatasetUtils.splitter(train_df=train_df, 
                               image_paths=feature_columns,targets='label')

In [14]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((703, 5), (703,), (235, 5), (235,))

In [18]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __getitem__(self, idx):
        return torch.tensor(self.features[idx]), torch.tensor(self.labels[idx])
    
    def __len__(self):
        return len(self.features)

In [19]:
train_dataset = MyDataset(X_train, y_train)
valid_dataset = MyDataset(X_test, y_test)

In [21]:
train_dataset.__getitem__(100)

(tensor([0.2370, 0.0488, 0.9920, 0.9749, 0.9874], dtype=torch.float64),
 tensor(0))

In [22]:
model =  Net(3)

In [23]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [24]:
loss = nn.CrossEntropyLoss()

In [25]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16)

In [29]:
?CustomTrainer

Init signature:
CustomTrainer(
    train_loader,
    val_loader,
    device,
    criterion,
    optimizer,
    model,
    epochs,
    metric_name,
    lr_scheduler=None,
    checkpoint_path='models',
    accumulation_steps=1,
)
Docstring:     
Abstract base class for defining your own custom training and validation steps,
basically your own `Trainer`.
File:           e:\library\myvision\engine\engine.py
Type:           ABCMeta
Subclasses:     


In [30]:
trainer = CustomTrainer(train_loader=train_loader,
                       val_loader=valid_loader,
                       device='cpu',
                       criterion=loss,
                       optimizer=optimizer,
                       model=model,
                       epochs=10,
                       metric_name='accuracy')

In [36]:
def train():
    pass

In [37]:
trainer.train(train)

TypeError: train() takes 0 positional arguments but 1 was given